In [1]:
import pandas as pd
import numpy as np

# Generate synthetic monthly time series data
date_range = pd.date_range(start='2018-01-01', end='2024-12-01', freq='MS')
data = np.sin(np.arange(len(date_range))) + np.random.normal(scale=0.1, size=len(date_range))
df = pd.DataFrame(data, index=date_range, columns=['value'])

# Display the first few rows
print(df.head())

               value
2018-01-01 -0.061179
2018-02-01  0.876786
2018-03-01  1.162192
2018-04-01  0.115444
2018-05-01 -0.755542


In [2]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

# Create sequences for training
def create_sequences(data, time_steps=12):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 12
X, y = create_sequences(scaled_data, time_steps)


In [3]:
import tensorflow as tf

# Convert to TensorFlow Dataset
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Build the GRU model
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(time_steps, 1)),
    GRU(50),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
epochs = 100
model.fit(dataset, epochs=epochs, verbose=1)


Epoch 1/100
3/3 [==============================] - 2s 9ms/step - loss: 0.3982
Epoch 2/100
3/3 [==============================] - 0s 9ms/step - loss: 0.2271
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.1255
Epoch 4/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1043
Epoch 5/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1362
Epoch 6/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1346
Epoch 7/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1157
Epoch 8/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1059
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 0.1048
Epoch 10/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1054
Epoch 11/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1046
Epoch 12/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1024
Epoch 13/100
3/3 [==============================] - 0s 6ms/st

In [7]:
# Prepare the input data for forecasting the next 12 months
forecast_input = scaled_data[-time_steps:].reshape(1, time_steps, 1)

# Forecast the next 12 months
forecasts = []
for _ in range(12):
    prediction = model.predict(forecast_input)
    forecasts.append(prediction[0, 0])
    
    # Update forecast_input: remove the first element and append the new prediction
    new_input = np.append(forecast_input[:, 1:, :], [[[prediction[0, 0]]]], axis=1)

    # Reshape the new input to maintain the correct dimensions
    forecast_input = new_input.reshape(1, time_steps, 1)

# Inverse transform the forecasts
forecasts = np.array(forecasts).reshape(-1, 1)
forecasts_original_scale = scaler.inverse_transform(forecasts)

# Display the forecasts
forecast_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=12, freq='MS')
forecast_df = pd.DataFrame(forecasts_original_scale, index=forecast_dates, columns=['forecast'])
print(forecast_df)


            forecast
2025-01-01  0.741145
2025-02-01 -0.145292
2025-03-01 -0.937983
2025-04-01 -0.839534
2025-05-01  0.034328
2025-06-01  0.838034
2025-07-01  0.899437
2025-08-01  0.168844
2025-09-01 -0.746457
2025-10-01 -0.964375
2025-11-01 -0.270747
2025-12-01  0.635835
